# ARIMA와 SARIMA 모델을 이용한 시계열 예측

이 노트북은 ARIMA(AutoRegressive Integrated Moving Average)와 SARIMA(Seasonal ARIMA) 모델을 이용하여 시계열 데이터를 분석하고 예측하는 과정을 보여줍니다.
ARIMA 모델은 시계열 데이터의 자기상관성(Autocorrelation)과 이동평균(Moving Average) 특성을 활용하며, 비정상성(Non-stationary) 데이터를 차분(Differencing)을 통해 정상성으로 변환하여 모델링합니다.
SARIMA 모델은 ARIMA 모델에 계절성(Seasonality) 요인을 추가하여 주기적인 패턴을 가진 시계열 데이터 분석에 더 적합합니다.

## 목표
1. 시계열 데이터 탐색 및 시각화를 통해 데이터 특성 파악
2. 정상성 검정 및 시계열 변환 (차분)을 통한 모델링 준비
3. ACF/PACF 분석을 통한 ARIMA/SARIMA 모델 파라미터(p, d, q, P, D, Q, m) 추정
4. ARIMA 및 SARIMA 모델 구축 및 학습
5. 모델 성능 평가 (잔차 분석 포함) 및 예측
6. 미래 예측 및 결과 분석 (신뢰 구간 포함)

## 1. 사용자 입력 파라미터 설정
분석 대상 데이터와 모델 파라미터를 사용자의 환경과 목적에 맞게 설정합니다.
- **데이터 파라미터**: 데이터 파일 경로, 날짜/타겟 컬럼명, 훈련 데이터 비율, 데이터 주기 등
- **ARIMA 파라미터 (비계절성)**: p(AR 차수), d(차분 차수), q(MA 차수)
- **SARIMA 파라미터 (계절성)**: 계절성 사용 여부(SEASONAL), P(계절성 AR), D(계절성 차분), Q(계절성 MA), m(계절성 주기 - SEASONAL_PERIOD)
- **예측 파라미터**: 미래 예측 기간(FORECAST_HORIZON)
- **기타 파라미터**: 랜덤 시드(RANDOM_STATE)
- **출력 경로**: 결과 저장 경로

In [ ]:
# ===== 데이터 파라미터 =====
# 데이터 파일 경로 (CSV 형식 권장)
DATA_PATH = '../data/raw/your_data.csv'  # 실제 데이터 파일 경로를 지정하세요

# 시간 열 이름 (데이터프레임 내 날짜/시간 정보가 있는 열)
DATE_COL = 'date'  # 실제 데이터의 날짜 열 이름을 지정하세요

# 타겟 열 이름 (예측하려는 변수가 있는 열)
TARGET_COL = 'value'  # 실제 데이터의 타겟 열 이름을 지정하세요

# 데이터 분할 비율
TRAIN_SIZE = 0.7                         # 훈련 데이터 비율 (기존 0.8에서 수정)
VALIDATION_SIZE = 0.15                   # 검증 데이터 비율 추가

# 시계열 데이터 주기 설정
FREQUENCY = 'D'  # D: 일별, M: 월별, Y: 연별, H: 시간별 등 (pandas frequency 문자열)

# ===== ARIMA 모델 파라미터 =====
# Auto ARIMA 사용 여부
USE_AUTO_ARIMA = False # True로 설정 시 아래 p, d, q, P, D, Q 값 대신 auto_arima가 찾은 최적 파라미터 사용

# ARIMA 파라미터 (비계절성) - USE_AUTO_ARIMA=False일 때 사용
P = 1  # AR 차수
D = 1  # 차분 차수
Q = 1  # MA 차수

# ===== SARIMA 모델 파라미터 =====
# 계절성 사용 여부 - USE_AUTO_ARIMA=False일 때도 적용될 수 있음 (auto_arima가 계절성 판단)
SEASONAL = True

# 계절성 ARIMA 파라미터 - USE_AUTO_ARIMA=False일 때 사용
SEASONAL_P = 1  # 계절성 AR 차수
SEASONAL_D = 1  # 계절성 차분 차수
SEASONAL_Q = 1  # 계절성 MA 차수
SEASONAL_PERIOD = 12 # 계절성 주기

# ===== 예측 파라미터 =====
# 미래 예측 기간
FORECAST_HORIZON = 12  # 미래 예측 스텝 수 (기존 FORECAST_STEPS에서 이름 변경)

# ===== 기타 파라미터 =====
RANDOM_STATE = 42      # 랜덤 시드 (재현성을 위해 추가, 현재 모델은 직접 사용 안함)

# ===== 출력 경로 설정 =====
# 결과 저장 경로
OUTPUT_DIR = '../plots'  # 시각화, 모델 결과를 저장할 경로
DATA_OUTPUT_DIR = '../data/processed'  # 데이터 파일을 저장할 경로

# 경로가 없는 경우 생성
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DATA_OUTPUT_DIR, exist_ok=True)

## 2. 필요 라이브러리 가져오기 및 데이터 탐색
시계열 분석, 모델링, 시각화 등에 필요한 라이브러리를 불러오고 기본 설정을 수행합니다.

In [ ]:
# 필요 라이브러리 가져오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
import pmdarima as pm # auto_arima를 위해 추가

# 경고 무시
warnings.filterwarnings('ignore')

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set(style="darkgrid")

# 한글 폰트 설정 (한글 출력이 필요한 경우)
try:
    plt.rc('font', family='Malgun Gothic')
    plt.rc('axes', unicode_minus=False)
except:
    pass

### 2.1 데이터 로드 및 전처리
지정된 경로에서 데이터를 로드하고 기본적인 정보(형태, 타입, 통계량, 샘플)를 확인합니다.
이 단계에서는 데이터 자체의 구조를 파악하는 데 중점을 둡니다.

In [ ]:
# 데이터 로드
# CSV 파일이 존재하지 않는 경우 명확한 에러 메시지 출력
print(f"데이터 파일 '{DATA_PATH}'을 로드합니다...")

# 데이터 로드 시도
df = pd.read_csv(DATA_PATH)
print(f"데이터 로드 완료: {df.shape[0]}행 {df.shape[1]}열")

# 데이터 정보 확인
print("\n데이터 정보:")
print(df.info())

print("\n데이터 통계:")
print(df.describe())

print("\n데이터 샘플:")
print(df.head())

### 2.2 시계열 데이터 변환
로드된 데이터를 시계열 분석에 적합한 형태로 변환합니다.
- **날짜 인덱스 설정**: 시간 순서 분석을 위해 날짜 컬럼을 인덱스로 변환합니다.
- **정렬**: 시간 순서가 올바른지 확인하고 정렬합니다.
- **타겟 변수 추출**: 분석할 특정 시계열 데이터를 선택합니다.
- **결측치 처리**: 분석 모델에 영향을 줄 수 있는 결측치를 보간합니다.

In [ ]:
# 날짜 열을 인덱스로 변환
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df.set_index(DATE_COL, inplace=True)
print(f"날짜 인덱스 변환 완료: {df.index.min()} ~ {df.index.max()}")

# 날짜순으로 정렬
df.sort_index(inplace=True)

# 선택한 타겟 변수 추출
ts_data = df[TARGET_COL]
print(f"타겟 변수 '{TARGET_COL}' 추출 완료, 데이터 포인트: {len(ts_data)}")

# 시계열 데이터 결측치 확인
missing_values = ts_data.isnull().sum()
if missing_values > 0:
    print(f"결측치 개수: {missing_values}")
    # 결측치 보간 (선형 보간법)
    ts_data = ts_data.interpolate(method='linear')
    print("결측치 선형 보간 완료")

### 2.3 시계열 데이터 시각화
변환된 시계열 데이터를 시각화하여 전반적인 패턴(추세, 계절성 등)과 분포를 확인합니다.
- **선 그래프**: 시간에 따른 타겟 변수의 변화 추이를 보여줍니다.
- **분포 플롯 (히스토그램, Q-Q 플롯)**: 타겟 변수 값의 분포 형태와 정규성 가정을 시각적으로 검토합니다.

In [ ]:
# 시계열 데이터 시각화
plt.figure(figsize=(14, 7))
plt.plot(ts_data, label=TARGET_COL)
plt.title(f'{TARGET_COL} 시계열 데이터')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.grid(True, alpha=0.3)
plt.legend()
plt.savefig(f'{OUTPUT_DIR}/arima_시계열데이터.png')
plt.show()

# 분포 확인
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
sns.histplot(ts_data, kde=True)
plt.title(f'{TARGET_COL} 분포')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
from scipy import stats
stats.probplot(ts_data, plot=plt)
plt.title('Q-Q 플롯')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_시계열_분포.png')
plt.show()

## 3. 정상성 확인 및 변환
ARIMA/SARIMA 모델은 입력 데이터가 정상성(Stationarity)을 만족할 때 더 안정적인 성능을 보입니다. 따라서 데이터의 정상성 여부를 확인하고, 필요시 차분을 통해 정상성을 확보합니다.
- **ACF/PACF 분석**: 정상성 여부 및 ARIMA 모델의 p, q 파라미터 추정을 위해 자기상관/부분자기상관 구조를 시각적으로 확인합니다.
- **정상성 검정 (ADF Test)**: 통계적 가설 검정을 통해 정상성 여부를 객관적으로 판단합니다.
- **차분 (Differencing)**: 비정상 시계열의 경우, 차분을 적용하여 평균이나 분산을 안정화시켜 정상성을 확보합니다. 계절성이 있는 경우 계절성 차분도 고려합니다.

### 3.1 자기상관 및 부분 자기상관 분석
ACF(Autocorrelation Function)는 현재 시점과 과거 시점들 간의 상관관계를, PACF(Partial Autocorrelation Function)는 다른 시점의 영향을 제외한 현재와 특정 과거 시점 간의 순수한 상관관계를 보여줍니다. 이를 통해 AR(p) 및 MA(q) 모델의 차수를 결정하는 데 도움을 받을 수 있습니다.
- **ACF**: 시계열의 MA(q) 차수를 추정하는 데 사용됩니다. ACF가 q 시점 이후 급격히 감소하면 MA(q) 모델을 고려합니다.
- **PACF**: 시계열의 AR(p) 차수를 추정하는 데 사용됩니다. PACF가 p 시점 이후 급격히 감소하면 AR(p) 모델을 고려합니다.

In [ ]:
# ACF 및 PACF 플롯
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plot_acf(ts_data, lags=20, alpha=0.05, ax=plt.gca())
plt.title('자기상관함수 (ACF)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plot_pacf(ts_data, lags=20, alpha=0.05, method='ywm', ax=plt.gca())
plt.title('부분 자기상관함수 (PACF)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_ACF_PACF.png')
plt.show()

### 3.2 정상성 검정
통계적 검정 방법을 사용하여 시계열 데이터의 정상성 여부를 판단합니다.
- **ADF 검정**: 귀무가설(H0)은 "시계열에 단위근이 존재한다(비정상성이다)"입니다. p-value가 유의수준(보통 0.05)보다 작으면 귀무가설을 기각하고 시계열이 정상적이라고 판단합니다.
- **KPSS 검정 (주석 처리됨)**: 귀무가설(H0)은 "시계열이 정상성이다"입니다. ADF와 귀무가설이 반대이므로 교차 검증에 활용될 수 있으나, 특정 데이터에서 오류 발생 가능성이 있어 여기서는 주석 처리했습니다.

In [ ]:
# ADF 검정 (귀무가설: 단위근이 존재한다 = 정상성이 아니다)
adf_result = adfuller(ts_data.dropna())
print('ADF 검정 결과:')
print(f'ADF 통계량: {adf_result[0]:.4f}')
print(f'p-value: {adf_result[1]:.4f}')
print(f'임계값:')
for key, value in adf_result[4].items():
    print(f'\t{key}: {value:.4f}')

# ADF 검정 결과 해석
print('\nADF 검정 결과 해석:')
if adf_result[1] < 0.05:
    print('귀무가설 기각 (p < 0.05): 시계열이 정상적일 가능성이 높습니다.')
    is_stationary = True
else:
    print('귀무가설 채택 (p >= 0.05): 시계열이 비정상적일 가능성이 높습니다.')
    is_stationary = False

# # KPSS 검정 (주석 처리됨)
# # 참고: KPSS 검정은 특정 데이터에서 오류를 발생시킬 수 있어 주석 처리합니다.
# #      필요시 주석을 해제하고 사용하세요.
# try:
#     kpss_result = kpss(ts_data.dropna(), regression='c')
#     print('\nKPSS 검정 결과:')
#     print(f'KPSS 통계량: {kpss_result[0]:.4f}')
#     print(f'p-value: {kpss_result[1]:.4f}')
#     print(f'임계값:')
#     for key, value in kpss_result[3].items():
#         print(f'\t{key}: {value:.4f}')
    
#     # KPSS 검정 결과 해석
#     print('\nKPSS 검정 결과 해석:')
#     if kpss_result[1] < 0.05:
#         print('귀무가설 기각: 시계열이 비정상적입니다.')
#     else:
#         print('귀무가설 채택: 시계열이 정상적입니다.')
# except Exception as e:
#     print(f"KPSS 검정 중 오류가 발생했습니다: {str(e)}")

### 3.3 차분 및 변환
ADF 검정 결과 시계열이 비정상적일 경우, 차분을 적용하여 정상성을 확보합니다. 일반 차분은 추세를 제거하고, 계절성 차분은 계절성을 제거하는 데 사용됩니다. (`D` 및 `SEASONAL_D` 파라미터로 차수 조절)
차분 후에도 ADF 검정 및 ACF/PACF 플롯을 통해 정상성 확보 여부를 확인합니다.

In [ ]:
# 차분
diff_data = ts_data.diff(D).dropna()
print(f"{D}차 차분 데이터 생성 완료")

# 차분 데이터 시각화
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
plt.plot(ts_data, label='원본 데이터')
plt.title('원본 시계열 데이터')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(diff_data, label=f'{D}차 차분')
plt.title(f'{D}차 차분 데이터')
plt.xlabel('날짜')
plt.ylabel(f'차분({TARGET_COL})')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_시계열_차분.png')
plt.show()

# 차분 데이터의 ACF 및 PACF 플롯
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plot_acf(diff_data, lags=20, alpha=0.05, ax=plt.gca())
plt.title(f'{D}차 차분 데이터의 ACF')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plot_pacf(diff_data, lags=20, alpha=0.05, method='ywm', ax=plt.gca())
plt.title(f'{D}차 차분 데이터의 PACF')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_차분_ACF_PACF.png')
plt.show()

# 차분 후 정상성 검정
adf_diff_result = adfuller(diff_data.dropna())
print(f'\n{D}차 차분 후 ADF 검정 결과:')
print(f'ADF 통계량: {adf_diff_result[0]:.4f}')
print(f'p-value: {adf_diff_result[1]:.4f}')

if adf_diff_result[1] < 0.05:
    print('차분 후 시계열이 정상적일 가능성이 높습니다 (p < 0.05).')
    diff_is_stationary = True
else:
    print('차분 후에도 시계열이 비정상적일 가능성이 높습니다 (p >= 0.05). 추가 차분이 필요할 수 있습니다.')
    diff_is_stationary = False

# 계절성 차분 (필요한 경우)
if SEASONAL and SEASONAL_D > 0:
    # 계절성 차분
    seasonal_diff_data = ts_data.diff(SEASONAL_PERIOD * SEASONAL_D).dropna()
    print(f"계절성 차분(주기={SEASONAL_PERIOD}, 차수={SEASONAL_D}) 데이터 생성 완료")
    
    # 계절성 + 일반 차분
    if D > 0:
        combined_diff_data = seasonal_diff_data.diff(D).dropna()
        print(f"계절성 차분 + {D}차 차분 데이터 생성 완료")
    else:
        combined_diff_data = seasonal_diff_data
    
    # 계절성 차분 데이터 시각화
    plt.figure(figsize=(14, 10))
    
    plt.subplot(3, 1, 1)
    plt.plot(ts_data, label='원본 데이터')
    plt.title('원본 시계열 데이터')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 1, 2)
    plt.plot(seasonal_diff_data, label='계절성 차분')
    plt.title(f'계절성 차분 데이터 (주기={SEASONAL_PERIOD}, 차수={SEASONAL_D})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 1, 3)
    plt.plot(combined_diff_data, label='계절성 + 일반 차분')
    plt.title(f'계절성 + 일반 차분 데이터')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/arima_계절성차분.png')
    plt.show()
    
    # 계절성 차분 후 정상성 검정
    adf_seasonal_diff_result = adfuller(combined_diff_data.dropna())
    print(f'\n계절성 + 일반 차분 후 ADF 검정 결과:')
    print(f'ADF 통계량: {adf_seasonal_diff_result[0]:.4f}')
    print(f'p-value: {adf_seasonal_diff_result[1]:.4f}')
    
    if adf_seasonal_diff_result[1] < 0.05:
        print('계절성 + 일반 차분 후 시계열이 정상적일 가능성이 높습니다 (p < 0.05).')
        seasonal_diff_is_stationary = True
    else:
        print('계절성 + 일반 차분 후에도 시계열이 비정상적일 가능성이 높습니다 (p >= 0.05).')
        seasonal_diff_is_stationary = False

## 4. 모델 파라미터 식별 및 모델 구축
정상성 확인 및 ACF/PACF 분석 결과를 바탕으로 ARIMA/SARIMA 모델의 파라미터(p, d, q, P, D, Q, m)를 설정하고 모델을 구축합니다.
- **데이터 분할**: 모델 학습 및 평가를 위해 데이터를 훈련, 검증, 테스트 세트로 분할합니다.
- **모델 구축**: `statsmodels` 라이브러리의 `ARIMA`, `SARIMAX` 클래스를 사용하여 모델 객체를 생성합니다.

In [ ]:
# 데이터 분할 (훈련/검증/테스트)
train_val_split_idx = int(len(ts_data) * TRAIN_SIZE)
val_test_split_idx = int(len(ts_data) * (TRAIN_SIZE + VALIDATION_SIZE))

train_data = ts_data[:train_val_split_idx]
val_data = ts_data[train_val_split_idx:val_test_split_idx] # 검증 세트 생성
test_data = ts_data[val_test_split_idx:]

print(f"훈련 데이터: {train_data.shape}, 기간: {train_data.index.min()} ~ {train_data.index.max()}")
print(f"검증 데이터: {val_data.shape}, 기간: {val_data.index.min()} ~ {val_data.index.max()}") # 검증 데이터 정보 출력
print(f"테스트 데이터: {test_data.shape}, 기간: {test_data.index.min()} ~ {test_data.index.max()}")

# 훈련/검증/테스트 데이터 시각화
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(val_data.index, val_data.values, label='검증 데이터', color='orange') # 검증 데이터 시각화 추가
plt.plot(test_data.index, test_data.values, label='테스트 데이터', color='red')
plt.axvline(x=train_data.index[-1], color='gray', linestyle='--')
plt.axvline(x=val_data.index[-1], color='gray', linestyle='--') # 검증/테스트 분할선 추가
plt.title('훈련/검증/테스트 데이터 분할')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_훈련검증테스트분할.png') # 파일명 변경
plt.show()

### 4.1 (선택) Auto ARIMA를 이용한 최적 파라미터 탐색
`pmdarima` 라이브러리의 `auto_arima` 함수를 사용하면 AIC, BIC 등의 정보 기준을 최소화하는 최적의 (S)ARIMA 모델 파라미터를 자동으로 탐색할 수 있습니다.
`USE_AUTO_ARIMA` 파라미터를 True로 설정하면 이 기능이 활성화됩니다.

In [ ]:
if USE_AUTO_ARIMA:
    print("Auto ARIMA를 사용하여 최적 파라미터를 탐색합니다...")
    # auto_arima 모델 학습
    # seasonal=True로 설정하여 SARIMA 모델까지 고려, m은 계절성 주기
    # stepwise=True로 설정하여 효율적인 파라미터 탐색 수행
    auto_arima_model = pm.auto_arima(
        train_data, 
        start_p=1, start_q=1, 
        max_p=3, max_q=3, # 탐색할 p, q 최대값
        m=SEASONAL_PERIOD, # 계절성 주기
        seasonal=SEASONAL, # 계절성 고려 여부
        d=None, D=None, # 차분 차수는 자동 결정
        trace=True, # 탐색 과정 출력
        error_action='ignore',  # 오류 발생 시 무시
        suppress_warnings=True, # 경고 메시지 숨김
        stepwise=True # 단계적 탐색 수행
    )
    
    print("\nAuto ARIMA 모델 요약:")
    print(auto_arima_model.summary())
    
    # 최적 모델을 이후 단계에서 사용하기 위해 할당
    # auto_arima 결과는 SARIMAX 모델과 호환됩니다.
    model_fit = auto_arima_model
    best_order = auto_arima_model.order
    best_seasonal_order = auto_arima_model.seasonal_order
    print(f"\n최적 파라미터: order={best_order}, seasonal_order={best_seasonal_order}")

### 4.2 ARIMA/SARIMA 모델 수동 구축 및 학습 (Auto ARIMA 미사용 시)
`USE_AUTO_ARIMA`가 False일 경우, 사용자가 직접 설정한 파라미터(p, d, q, P, D, Q, m)를 사용하여 모델을 구축하고 학습합니다.

In [ ]:
if not USE_AUTO_ARIMA:

    # #### 4.2.1 ARIMA 모델 구축 및 학습
    print(f"수동 파라미터 ARIMA({P},{D},{Q}) 모델을 구축합니다.")
    model_arima = ARIMA(train_data, order=(P, D, Q))
    model_arima_fit = model_arima.fit()
    print("\nARIMA 모델 요약:")
    print(model_arima_fit.summary())
    model_fit = model_arima_fit # 이후 단계에서 사용할 모델

    # #### 4.2.2 SARIMA 모델 구축 및 학습 (계절성이 있는 경우)
    if SEASONAL:
        print(f"\n수동 파라미터 SARIMA({P},{D},{Q})x({SEASONAL_P},{SEASONAL_D},{SEASONAL_Q},{SEASONAL_PERIOD}) 모델을 구축합니다.")
        model_sarima = SARIMAX(
            train_data,
            order=(P, D, Q),
            seasonal_order=(SEASONAL_P, SEASONAL_D, SEASONAL_Q, SEASONAL_PERIOD)
        )
        model_sarima_fit = model_sarima.fit(disp=False)
        print("\nSARIMA 모델 요약:")
        print(model_sarima_fit.summary())
        # 계절성 모델이 우선순위가 높다고 가정하고 model_fit 업데이트
        # 필요시 모델 비교 후 최적 모델을 model_fit에 할당하도록 수정 가능
        model_fit = model_sarima_fit 

## 5. 모델 학습 및 예측
학습된 최적 모델 (`model_fit`: Auto ARIMA 결과 또는 수동 설정된 (S)ARIMA 결과)을 사용하여 예측을 수행합니다.

### 5.1 모델 예측 (훈련 및 테스트)
학습된 모델을 사용하여 훈련 기간과 테스트 기간에 대한 예측값을 생성하고 실제값과 비교하여 시각화합니다.

In [ ]:
# 훈련 데이터 예측
pred_train = model_fit.predict(start=0, end=len(train_data)-1)

# 테스트 데이터 예측
# Auto ARIMA 결과(SARIMAX)와 수동 ARIMA/SARIMAX 모두 forecast 메서드 사용 가능
pred_test = model_fit.predict(start=len(train_data), end=len(ts_data)-1) 
# pred_test = model_fit.forecast(steps=len(test_data)) # forecast도 가능
pred_test = pd.Series(pred_test, index=test_data.index)

# 시각화
model_name = "Auto ARIMA" if USE_AUTO_ARIMA else ("SARIMA" if SEASONAL and not USE_AUTO_ARIMA else "ARIMA")
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(test_data.index, test_data.values, label='테스트 데이터', color='red')
plt.plot(pred_train.index, pred_train.values, label='훈련 예측', color='green', linestyle='--')
plt.plot(pred_test.index, pred_test.values, label='테스트 예측', color='purple', linestyle='--')
plt.title(f'{model_name} 모델 예측')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/{model_name}_모델예측.png')
plt.show()

### 5.2 미래 예측 및 신뢰 구간
최종 모델을 사용하여 지정된 미래 기간(`FORECAST_HORIZON`)만큼 예측하고 신뢰 구간과 함께 시각화합니다.

In [ ]:
# 미래 예측
future_pred = model_fit.predict(n_periods=FORECAST_HORIZON) # auto_arima 결과 사용 시 predict
# future_pred = model_fit.forecast(steps=FORECAST_HORIZON) # statsmodels 결과 사용 시 forecast
future_dates = pd.date_range(start=ts_data.index[-1], periods=FORECAST_HORIZON+1, freq=FREQUENCY)[1:]
future_pred = pd.Series(future_pred, index=future_dates)

# 신뢰 구간 계산 (auto_arima는 predict에 포함, statsmodels는 get_forecast 사용)
if USE_AUTO_ARIMA:
    # auto_arima는 predict 호출 시 return_conf_int=True로 신뢰구간 얻음
    future_pred_ci = model_fit.predict(n_periods=FORECAST_HORIZON, return_conf_int=True)[1]
    lower_ci = pd.Series(future_pred_ci[:, 0], index=future_dates)
    upper_ci = pd.Series(future_pred_ci[:, 1], index=future_dates)
else:
    pred_ci = model_fit.get_forecast(steps=FORECAST_HORIZON).conf_int()
    lower_ci = pd.Series(pred_ci.iloc[:, 0].values, index=future_dates)
    upper_ci = pd.Series(pred_ci.iloc[:, 1].values, index=future_dates)

# 시각화
plt.figure(figsize=(14, 7))
plt.plot(ts_data.index, ts_data.values, label='실제 데이터', color='blue')
plt.plot(future_dates, future_pred.values, label='미래 예측', color='red', linestyle='--')
plt.fill_between(future_dates, lower_ci.values, upper_ci.values, color='pink', alpha=0.3, label='95% 신뢰 구간')
plt.title(f'{model_name} 모델 미래 예측')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/{model_name}_미래예측.png')
plt.show()

## 6. 모델 평가 및 예측 결과 분석
모델의 성능을 정량적으로 평가하고, 잔차 분석을 통해 모델 가정이 잘 만족되었는지 진단합니다.
- **성능 지표**: RMSE, MAE, R² 등을 계산하여 예측 오차의 크기와 설명력을 평가합니다.
- **잔차 분석**: 모델의 예측 오차(잔차)가 특정 패턴 없이 랜덤하게 분포하는지(백색 잡음 가설) 확인합니다. 잔차의 시계열 플롯, 분포(히스토그램), 자기상관(ACF 플롯) 등을 검토합니다. 잔차에 패턴이 남아있다면 모델이 데이터의 모든 정보를 포착하지 못했다는 의미일 수 있습니다.

### 6.1 모델 평가 (최종 모델 기준)
최종 선택된 모델(`model_fit`)의 테스트 세트에 대한 성능 지표를 계산하고 잔차 분석을 수행합니다.

In [ ]:
# 최종 모델 평가
final_mse = mean_squared_error(test_data, pred_test)
final_rmse = np.sqrt(final_mse)
final_mae = mean_absolute_error(test_data, pred_test)
final_r2 = r2_score(test_data, pred_test)

print(f"{model_name} 모델 평가:")
print(f"MSE: {final_mse:.4f}")
print(f"RMSE: {final_rmse:.4f}")
print(f"MAE: {final_mae:.4f}")
print(f"R²: {final_r2:.4f}")

# 잔차 분석 (statsmodels의 plot_diagnostics 사용)
# 참고: auto_arima 결과(pmdarima 객체)는 plot_diagnostics 메서드를 직접 가짐
#      statsmodels 결과는 fit된 객체에서 호출
plt.figure(figsize=(14, 10))
if hasattr(model_fit, 'plot_diagnostics'): # pmdarima 객체 확인
    model_fit.plot_diagnostics(fig=plt.gcf())
else: # statsmodels 객체
    # statsmodels의 plot_diagnostics는 새 figure를 생성하므로 plt.figure() 불필요
    # 대신, 생성된 figure를 가져와서 저장하거나 표시할 수 있음
    diag_fig = model_fit.plot_diagnostics(figsize=(14, 10))
    diag_fig.suptitle(f'{model_name} 모델 잔차 분석', y=1.02) # 제목 추가
    diag_fig.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/{model_name}_잔차분석.png') # 여기서 저장
    plt.show() # 위에서 저장했으므로 show만

### 6.2 모델 비교 (삭제 또는 Auto ARIMA 사용 시 불필요)
이전 버전에서는 ARIMA와 수동 SARIMA를 비교했지만, Auto ARIMA를 사용하거나 단일 모델만 구축하는 경우 이 섹션은 불필요할 수 있습니다.
필요하다면 수동 설정 모델과 Auto ARIMA 결과를 비교하는 로직을 추가할 수 있습니다.

### 6.3 결론 및 인사이트
... 